In [1]:
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount("/content/drive")

# Load the dataframe from 'df'
# Replace 'your_dataframe.csv' with the actual filename or path
df = pd.read_parquet("/content/drive/My Drive/datasets/dynahate_train.parquet.gzip")

# Filter explicit and implicit instances
explicit_df = df[df['implicit_layer'] == 'Explicit HS']
implicit_df = df[df['implicit_layer'] == 'Implicit HS']

# Create positive pairs (Explicit HS, Implicit HS)
positive_pairs = []
for _, implicit_row in implicit_df.iterrows():
    implicit_target = implicit_row['sanitized_target']

    # Find explicit instances with the same target as the implicit instance
    same_target_explicit = explicit_df[explicit_df['sanitized_target'] == implicit_target]

    # If there are explicit instances with the same target, randomly choose one
    if len(same_target_explicit) > 0:
        explicit_row = same_target_explicit.sample(n=1).iloc[0]
    else:
        # If there are no explicit instances with the same target, choose any random explicit instance
        explicit_row = explicit_df.sample(n=1).iloc[0]

    # Append positive pair to the list
    positive_pairs.append((explicit_row['text'], implicit_row['text']))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
positive_pairs

[("next time i see a trans person, i'm going to beat them up",
  "I'm not saying we should actually eliminate heebs.. I just wish they naturally became extinct"),
 ("I can't tell the difference between a letterbox and a Muslim women",
  "It's such a rape culture with these immigrants, women don't even see it as rape"),
 ('I can tell the difference between a Muslim women and a letterbox: the latter at least is useful!',
  "I'm not saying we should actually eliminate kikes.. I just wish they naturally became extinct"),
 ("You're either a man or a woman, anything else is a cry for attention",
  'Immigrants have a culture of rape'),
 ('You can´t mix Black and White people, it is just wrong',
  "I'm not saying we should actually eliminate Ikey-mos.. I just wish they naturally became extinct"),
 ('There are too many immigrants now and something needs to be done about this',
  'Foreigners have a culture of rape'),
 ('all gays are pervs', 'Blacks have a culture of rape'),
 ('it is true that le

In [3]:
non_hs_df = df[df['label'] == 0]
hs_df = df[df['label'] == 1]

In [4]:
hs_df

,text,implicit_layer,label,sanitized_target,target
0,dalits are lowlives,Explicit HS,1,WOMEN,WOMEN
1,gay people are a burden to our society,Explicit HS,1,NOTGIVEN,notgiven
2,Arabs are not welcome,Explicit HS,1,NOTGIVEN,notgiven
3,I'm not saying we should actually eliminate he...,Implicit HS,1,NOTGIVEN,notgiven
4,It's such a rape culture with these immigrants...,Implicit HS,1,NOTGIVEN,notgiven
...,...,...,...,...,...
32917,Women who don't want kids are freaks,Implicit HS,1,NOTGIVEN,notgiven
32918,Men own women,Explicit HS,1,NOTGIVEN,notgiven
32919,Women are deceitful,Explicit HS,1,NOTGIVEN,notgiven
32920,"Women don't want equality, they just want to b...",Explicit HS,1,WOMEN,WOMEN


In [5]:
non_hs_df = non_hs_df.sample(5000)
hs_df = hs_df.sample(5000)

In [6]:
# Create negative pairs (HS, Non-HS)
negative_pairs = []

for _, non_hs_row in non_hs_df.iterrows():
    non_hs_message = non_hs_row['text']

    # Randomly choose a HS message as the negative pair
    hs_message = np.random.choice(hs_df["text"])

    # Append negative pair to the list
    negative_pairs.append((hs_message, non_hs_message))

# Now you have positive_pairs and negative_pairs, which contain the positive and negative pairs respectively.
# You can use these pairs to train your contrastive learning model.
# The format of each pair is (message1, message2)

In [7]:
len(negative_pairs)

5000

In [8]:
!pip install torch
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import torch
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
from torch import nn

from huggingface_hub import login
from transformers import AutoModel, AutoTokenizer

login(token="YOUR_TOKEN")


# Load BERT model and tokenizer
model_name = "YOUR_USER/model-hatebert__trained-in-dynahate__seed-0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Some weights of the model checkpoint at BenjaminOcampo/model-bert__trained-in-dynahate__seed-0 were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
class ContrastiveDataset(Dataset):
    def __init__(self, pairs, labels, tokenizer):
        self.pairs = pairs
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        message1, message2 = self.pairs[idx]

        encoding1 = self.tokenizer(message1, truncation=True, padding="max_length", max_length=512, return_tensors="pt")
        encoding2 = self.tokenizer(message2, truncation=True, padding="max_length", max_length=512, return_tensors="pt")

        input_ids1 = encoding1["input_ids"].squeeze()
        attention_mask1 = encoding1["attention_mask"].squeeze()
        input_ids2 = encoding2["input_ids"].squeeze()
        attention_mask2 = encoding2["attention_mask"].squeeze()
        labels = self.labels[idx]

        return input_ids1, attention_mask1, input_ids2, attention_mask2, labels

In [11]:
class ContrastiveModel(nn.Module):
    def __init__(self, model):
        super(ContrastiveModel, self).__init__()
        self.model = model
        self.embedding_dim = model.config.hidden_size
        self.fc = nn.Linear(self.embedding_dim, self.embedding_dim)

    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids, attention_mask)
        embeddings = outputs.last_hidden_state[:, 0]  # Use the CLS token embedding as the representation

        return self.fc(embeddings)

In [12]:
batch_size = 8  # Adjust the batch size as needed

# Combine positive and negative pairs
all_pairs = positive_pairs + negative_pairs

labels = torch.tensor([1] * len(positive_pairs) + [0] * len(negative_pairs))

# Create the contrastive dataset
dataset = ContrastiveDataset(all_pairs, labels, tokenizer)

# Set the batch size and create the data loader
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [13]:
class ContrastiveLoss(nn.Module):
    def __init__(self, margin):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin
        self.cosine_similarity = nn.CosineSimilarity(dim=1)

    def forward(self, embeddings1, embeddings2, target):
        similarity = self.cosine_similarity(embeddings1, embeddings2)
        loss = torch.mean((1 - target) * torch.pow(similarity, 2) +
                          target * torch.pow(torch.clamp(self.margin - similarity, min=0.0), 2))
        return loss

In [14]:
contrastive_model = ContrastiveModel(model)
contrastive_loss = ContrastiveLoss(margin=0.5)  # Adjust the margin as needed

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available

contrastive_model.to(device)
contrastive_model.train()

optimizer = torch.optim.Adam(contrastive_model.parameters(), lr=1e-5)  # Adjust the learning rate as needed
num_epochs = 4  # Adjust the number of epochs as needed

for epoch in range(num_epochs):
    total_loss = 0.0

    for batch in data_loader:
        input_ids1, attention_mask1, input_ids2, attention_mask2, labels = batch

        input_ids1 = input_ids1.to(device)
        attention_mask1 = attention_mask1.to(device)
        input_ids2 = input_ids2.to(device)
        attention_mask2 = attention_mask2.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        embeddings1 = contrastive_model(input_ids1, attention_mask1)
        embeddings2 = contrastive_model(input_ids2, attention_mask2)

        loss = contrastive_loss(embeddings1, embeddings2, labels)
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {total_loss:.4f}")

Epoch 1/4 - Loss: 29.5046
Epoch 2/4 - Loss: 13.4742
